<a href="https://colab.research.google.com/github/dmarx/notebooks/blob/text2video-modelscope/text_to_video_synthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Setup

!apt -y install -qq aria2
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/text-to-video-synthesis/resolve/main/hub/damo/text-to-video-synthesis/VQGAN_autoencoder.pth -d /content/models -o VQGAN_autoencoder.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/text-to-video-synthesis/resolve/main/hub/damo/text-to-video-synthesis/open_clip_pytorch_model.bin -d /content/models -o open_clip_pytorch_model.bin
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/text-to-video-synthesis/resolve/main/hub/damo/text-to-video-synthesis/text2video_pytorch_model.pth -d /content/models -o text2video_pytorch_model.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/text-to-video-synthesis/raw/main/hub/damo/text-to-video-synthesis/configuration.json -d /content/models -o configuration.json

# from https://huggingface.co/kabachuha/modelscope-damo-text2video-pruned-weights
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/kabachuha/modelscope-damo-text2video-pruned-weights/resolve/main/VQGAN_autoencoder.pth -d /content/models -o VQGAN_autoencoder.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/kabachuha/modelscope-damo-text2video-pruned-weights/resolve/main/open_clip_pytorch_model.bin -d /content/models -o open_clip_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/kabachuha/modelscope-damo-text2video-pruned-weights/resolve/main/text2video_pytorch_model.pth -d /content/models -o text2video_pytorch_model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/kabachuha/modelscope-damo-text2video-pruned-weights/raw/main/configuration.json -d /content/models -o configuration.json

!pip install -q open_clip_torch pytorch_lightning 
!pip install -q git+https://github.com/dmarx/modelscope
!sed -i -e 's/\"tiny_gpu\": 1/\"tiny_gpu\": 0/g' /content/models/configuration.json

import os
os._exit(0)

E: dpkg was interrupted, you must manually run 'dpkg --configure -a' to correct the problem. 

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
c922df|OK  |       0B/s|/content/models/VQGAN_autoencoder.pth

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
7b238e|OK  |   260MiB/s|/content/models/open_clip_pytorch_model.bin

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
6218ad|OK  |   318MiB/s|/content/models/text2video_pytorch_model.pth

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
13ab55|OK  |   1.0KiB/s|/content/models/configuration.js

In [1]:
import torch, random, gc
from modelscope.pipelines import pipeline
from modelscope.outputs import OutputKeys
from pathlib import Path

torch.manual_seed(random.randint(0, 2147483647))
pipe = pipeline('text-to-video-synthesis', '/content/models')

outdir = Path('videos')
outdir.mkdir(exist_ok=True)
#!mkdir /content/videos


2023-03-28 16:06:59,673 - modelscope - INFO - PyTorch version 1.13.1+cu116 Found.
2023-03-28 16:06:59,681 - modelscope - INFO - TensorFlow version 2.11.0 Found.
2023-03-28 16:06:59,682 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2023-03-28 16:06:59,718 - modelscope - INFO - No valid ast index found from /root/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2023-03-28 16:06:59,829 - modelscope - INFO - Loading done! Current index file version is 1.4.1, with md5 5c5670cbd21f0405a8906ee3567b8a1e and a total number of 844 components indexed
2023-03-28 16:07:03,293 - modelscope - INFO - initiate model from /content/models
2023-03-28 16:07:03,295 - modelscope - INFO - initiate model from location /content/models.
2023-03-28 16:07:03,297 - modelscope - INFO - initialize model from /content/models
2023-03-28 16:07:39,143 - modelscope - WARNING - No preprocessor field found in cfg.
2023-03-28 16:07:39,145 - modelscope - WARNING - No val ke

In [8]:
pipe.model.config.model.model_args.max_frames = 64

In [9]:
import gc
import datetime
from IPython.display import HTML

with torch.no_grad():
  torch.cuda.empty_cache()
gc.collect()

test_text = {
        'text': 'A panda eating bamboo on a rock.',
    }
output_video_path = pipe(test_text,)[OutputKeys.OUTPUT_VIDEO]

new_video_path = f'/content/videos/{datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")}.mp4'
!ffmpeg -y -i {output_video_path} -c:v libx264 -c:a aac -strict -2 {new_video_path} >/dev/null 2>&1

print(output_video_path, '->', new_video_path)

2023-03-28 16:11:03,201 - modelscope - WARNING - task text-to-video-synthesis input definition is missing
2023-03-28 16:11:43,925 - modelscope - WARNING - task text-to-video-synthesis output keys are missing


/tmp/tmpkmme_wcb.mp4 -> /content/videos/2023-03-28_16:11:43.mp4


In [10]:
from IPython.display import HTML
from base64 import b64encode

!cp {new_video_path} /content/videos/tmp.mp4
mp4 = open('/content/videos/tmp.mp4','rb').read()

decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src="{decoded_vid}" type="video/mp4"></video>')

In [3]:
from IPython.display import HTML
from base64 import b64encode

!cp {new_video_path} /content/videos/tmp.mp4
mp4 = open('/content/videos/tmp.mp4','rb').read()

decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src="{decoded_vid}" type="video/mp4"></video>')